In [ ]:
!pip install streamlit bcrypt PyJWT pyngrok textstat PyPDF2

In [ ]:
%%writefile config.py
import os

JWT_SECRET = os.getenv("JWT_SECRET_KEY")
JWT_ALGORITHM = "HS256"
TOKEN_EXPIRY_MINUTES = 60

MAX_LOGIN_ATTEMPTS = 3
LOCK_TIME_MINUTES = 5
PASSWORD_HISTORY_COUNT = 3

EMAIL_ID = os.getenv("EMAIL_ID")
EMAIL_APP_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")

ADMIN_EMAIL = os.getenv("ADMIN_EMAIL_ID")
ADMIN_PASSWORD = os.getenv("ADMIN_PASSWORD")

In [ ]:
%%writefile db.py
import sqlite3
import json
from datetime import datetime

DB_NAME = "policynav_users.db"

def get_connection():
    return sqlite3.connect(DB_NAME, check_same_thread=False)

def init_db():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        email TEXT UNIQUE,
        password_hash TEXT,
        security_question TEXT,
        security_answer_hash TEXT,
        failed_attempts INTEGER DEFAULT 0,
        lock_until TEXT,
        password_history TEXT
    )
    """)

    conn.commit()
    conn.close()

def create_user(username, email, password_hash, question, answer_hash):
    conn = get_connection()
    cursor = conn.cursor()
    try:
        history = json.dumps([password_hash])
        cursor.execute("""
        INSERT INTO users (username, email, password_hash, security_question,
                           security_answer_hash, password_history)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (username, email, password_hash, question, answer_hash, history))
        conn.commit()
        return True, "User created"
    except:
        return False, "User exists"
    finally:
        conn.close()

def get_user_by_email(email):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE email=?", (email,))
    user = cursor.fetchone()
    conn.close()
    return user

def update_login_attempts(email, attempts, lock_until=None):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE users SET failed_attempts=?, lock_until=? WHERE email=?",
        (attempts, lock_until, email)
    )
    conn.commit()
    conn.close()

def update_password(email, new_hash):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET password_hash=? WHERE email=?", (new_hash, email))
    conn.commit()
    conn.close()

def update_password_history(email, history_json):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE users SET password_history=? WHERE email=?",
        (history_json, email)
    )
    conn.commit()
    conn.close()

def get_all_users():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT username, email, failed_attempts, lock_until FROM users")
    rows = cursor.fetchall()
    conn.close()
    return rows

In [ ]:
%%writefile auth.py
import re
import bcrypt
import jwt
import random
import smtplib
from datetime import datetime, timedelta
from email.mime.text import MIMEText
from config import *

# ---------- HASHING ----------
def hash_text(text: str) -> str:
    return bcrypt.hashpw(text.encode(), bcrypt.gensalt()).decode()

def verify_text(text: str, hashed: str) -> bool:
    return bcrypt.checkpw(text.encode(), hashed.encode())

# ---------- EMAIL VALIDATION ----------
def validate_email(email: str) -> bool:
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

# ---------- PASSWORD VALIDATION ----------
def validate_password(password: str):
    if len(password) < 8:
        return False, "Password must be at least 8 characters"
    if not re.search(r"[A-Z]", password):
        return False, "Password must contain at least 1 uppercase letter"
    if not re.search(r"[a-z]", password):
        return False, "Password must contain at least 1 lowercase letter"
    if not re.search(r"\d", password):
        return False, "Password must contain at least 1 number"
    return True, "Valid password"

# ---------- JWT ----------
def generate_token(email: str):
    payload = {
        "email": email,
        "exp": datetime.utcnow() + timedelta(minutes=TOKEN_EXPIRY_MINUTES)
    }
    return jwt.encode(payload, JWT_SECRET, algorithm=JWT_ALGORITHM)

def verify_token(token: str):
    try:
        return jwt.decode(token, JWT_SECRET, algorithms=[JWT_ALGORITHM])
    except:
        return None

# ---------- OTP ----------
def generate_otp():
    return str(random.randint(100000, 999999))

def send_otp_email(receiver, otp):
    html_content = f"""
    <html>
    <body style="font-family: Arial, sans-serif; background-color: #f4f4f4; padding: 20px;">
        <div style="
            max-width: 400px;
            margin: auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        ">
            <h2 style="color: #2c3e50;">PolicyNav Verification</h2>
            <p style="color: #555;">Use the OTP below to continue:</p>

            <div style="
                font-size: 28px;
                font-weight: bold;
                letter-spacing: 3px;
                color: #00b894;
                margin: 20px 0;
            ">
                {otp}
            </div>

            <p style="color: #999; font-size: 12px;">
                This OTP is valid for 10 minutes.
            </p>
        </div>
    </body>
    </html>
    """

    msg = MIMEText(html_content, "html")
    msg["Subject"] = "PolicyNav OTP Verification"
    msg["From"] = EMAIL_ID
    msg["To"] = receiver

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(EMAIL_ID, EMAIL_APP_PASSWORD)
        server.send_message(msg)

In [ ]:
%%writefile readability.py
import textstat

class ReadabilityAnalyzer:
    def __init__(self, text):
        self.text = text

    def get_all_metrics(self):
        return {
            "Flesch Reading Ease": textstat.flesch_reading_ease(self.text),
            "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(self.text),
            "SMOG Index": textstat.smog_index(self.text),
            "Gunning Fog": textstat.gunning_fog(self.text),
            "Coleman-Liau": textstat.coleman_liau_index(self.text)
        }

In [ ]:
%%writefile app.py
import plotly.graph_objects as go
import os
import PyPDF2
from readability import ReadabilityAnalyzer
import streamlit as st
import json
from datetime import datetime, timedelta
from db import *
from auth import *
from config import *
import base64

init_db()

st.set_page_config(page_title="PolicyNav Auth", layout="centered")

# ---------- BACKGROUND ----------
def set_bg(image_file="99.jpg"):
    if os.path.exists(image_file):
        with open(image_file, "rb") as f:
            encoded = base64.b64encode(f.read()).decode()

        bg_style = f"""
        <style>
        .stApp {{
            background: url("data:image/jpg;base64,{encoded}") no-repeat center center fixed;
            background-size: cover;
        }}
        </style>
        """
    else:
        bg_style = """
        <style>
        .stApp {
            background: linear-gradient(135deg, #000000, #1c1c1c);
            color: white;
        }
        </style>
        """

    st.markdown(bg_style, unsafe_allow_html=True)

set_bg()

# ---------- SESSION STATE ----------
if "token" not in st.session_state:
    st.session_state.token = None

if "page" not in st.session_state:
    st.session_state.page = "Login"

if "reset_stage" not in st.session_state:
    st.session_state.reset_stage = 0

# ---------- NAVIGATION ----------
def go_to(page):
    st.session_state.page = page
    st.rerun()

# ---------- LOGIN ----------
def login_page():
    st.title("PolicyNav – Login")

    with st.form("login_form"):

        email = st.text_input("Email")
        password = st.text_input("Password", type="password")

        left, center, right = st.columns([1.5, 1.5, 5])

        login_btn = left.form_submit_button("Login")
        signup_btn = center.form_submit_button("Signup")
        forgot_btn = right.form_submit_button("Forgot Password")

        # ---------- LOGIN ----------
        if login_btn:

            if not email:
                st.toast("Email required", icon="⚠️")
                return

            if not password:
                st.toast("Password required", icon="⚠️")
                return

            if email == ADMIN_EMAIL and password == ADMIN_PASSWORD:
                go_to("AdminDashboard")
                return

            user = get_user_by_email(email)
            if not user:
                st.toast("Email not registered", icon="❌")
                return

            failed_attempts = user[6]
            lock_until = user[7]

            if lock_until and datetime.utcnow() < datetime.fromisoformat(lock_until):
                st.toast("Account locked. Try again later.", icon="🔒")
                return

            if not verify_text(password, user[3]):

                failed_attempts += 1

                if failed_attempts >= MAX_LOGIN_ATTEMPTS:
                    lock_time = datetime.utcnow() + timedelta(minutes=LOCK_TIME_MINUTES)
                    update_login_attempts(email, failed_attempts, lock_time.isoformat())
                    st.toast("Account locked for 5 minutes", icon="🔒")
                else:
                    update_login_attempts(email, failed_attempts)
                    st.toast("Invalid credentials", icon="❌")

                return

            update_login_attempts(email, 0, None)

            otp = generate_otp()
            send_otp_email(email, otp)

            st.session_state.pending_email = email
            st.session_state.otp = otp
            st.session_state.otp_time = datetime.utcnow()

            go_to("OTP")

        # ---------- SIGNUP ----------
        if signup_btn:
            go_to("Signup")

        # ---------- FORGOT PASSWORD ----------
        if forgot_btn:
            go_to("Forgot")

# ---------- OTP ----------
def otp_page():
    st.title("OTP Verification")

    entered = st.text_input("Enter OTP")

    if st.button("Verify OTP"):

        if "otp_time" not in st.session_state:
            st.toast("OTP expired. Please login again.", icon="⚠️")
            go_to("Login")
            return

        # ⭐ Check 10-minute expiry
        if datetime.utcnow() - st.session_state.otp_time > timedelta(minutes=10):
            st.toast("OTP expired. Please login again.", icon="⏰")
            go_to("Login")
            return

        if entered == st.session_state.otp:

            # ⭐ Forgot password via OTP
          if st.session_state.get("flow") == "forgot_otp":
            go_to("SetNewPassword")

           # ⭐ Normal login OTP
          else:
            st.session_state.token = generate_token(st.session_state.pending_email)
            go_to("Dashboard")
        else:
            st.toast("Invalid OTP", icon="❌")

    if st.button("⬅ Back to Login"):
        go_to("Login")

# ---------- SIGNUP ----------
def signup_page():
    st.title("Signup")

    username = st.text_input("Username")
    email = st.text_input("Email")
    password = st.text_input("Password", type="password")
    confirm = st.text_input("Confirm Password", type="password")

    question = st.selectbox("Security Question", [
        "What is your favorite book?",
        "What is your dream job?"
    ])

    answer = st.text_input("Security Answer")

    if st.button("Create Account"):

        if not validate_email(email):
            st.toast("Invalid email format", icon="⚠️")
            return

        valid, msg = validate_password(password)
        if not valid:
            st.toast(msg, icon="⚠️")
            return

        if password != confirm:
            st.toast("Passwords do not match", icon="⚠️")
            return

        success, msg = create_user(
            username,
            email,
            hash_text(password),
            question,
            hash_text(answer)
        )

        if success:
            st.toast("Account created successfully", icon="✅")
            go_to("Login")
        else:
            st.toast(msg, icon="❌")

    if st.button("⬅ Back to Login"):
        go_to("Login")

# ---------- DASHBOARD ----------
def dashboard_page():
    payload = verify_token(st.session_state.token)

    if not payload:
        st.toast("Session expired", icon="❌")
        go_to("Login")

    user = get_user_by_email(payload["email"])

    # ⭐ LEFT SIDEBAR MENU
    st.sidebar.title("Navigation")

    if st.sidebar.button("📘 Readability Analyzer"):
        go_to("Readability")

    if st.sidebar.button("🔒 Reset Password"):
        go_to("Reset")

    if st.sidebar.button("🚪 Logout"):
        st.session_state.token = None
        go_to("Login")

    # ⭐ MAIN DASHBOARD CONTENT
    st.title(f"Welcome, {user[1]} 👋")
    st.write("Select an option from the left menu.")

# ---------- RESET ----------
def reset_page():
    st.title("🔒 Reset Password")

    payload = verify_token(st.session_state.token)

    if not payload:
        st.toast("Session expired", icon="❌")
        go_to("Login")

    user = get_user_by_email(payload["email"])

    old_password = st.text_input("Old Password", type="password")
    new_password = st.text_input("New Password", type="password")

    error = False

    if st.button("Update Password"):

      if not verify_text(old_password, user[3]):
        st.toast("Old password incorrect", icon="❌")
        error = True

      if not error:

        history = json.loads(user[8] or "[]")

        for old_hash in history:
            if verify_text(new_password, old_hash):
                st.toast("Cannot reuse old password", icon="⚠️")
                error = True
                break

      if not error:
        new_hash = hash_text(new_password)

        history.insert(0, new_hash)
        history = history[:PASSWORD_HISTORY_COUNT]

        update_password(payload["email"], new_hash)
        update_password_history(payload["email"], json.dumps(history))

        st.toast("Password updated", icon="✅")
        st.session_state.token = None
        go_to("Login")

        history = json.loads(user[8] or "[]")

        for old_hash in history:
            if verify_text(new_password, old_hash):
                st.toast("Cannot reuse old password", icon="⚠️")
                return

        new_hash = hash_text(new_password)

        history.insert(0, new_hash)
        history = history[:PASSWORD_HISTORY_COUNT]

        update_password(payload["email"], new_hash)
        update_password_history(payload["email"], json.dumps(history))

        st.toast("Password updated", icon="✅")
        st.session_state.token = None
        go_to("Login")

    if st.button("⬅ Back to Dashboard"):
        go_to("Dashboard")

# ---------- READABILITY PAGE ----------
def readability_page():
    st.title("📘 Text Readability Analyzer")

    payload = verify_token(st.session_state.token)

    if not payload:
        st.toast("Session expired", icon="❌")
        go_to("Login")

    option = st.radio("Choose Input Type", ["Enter Text", "Upload File"])

    text = ""

    if option == "Enter Text":
        text = st.text_area("Enter text to analyze")

    else:
        uploaded_file = st.file_uploader("Upload TXT or PDF", type=["txt", "pdf"])

        if uploaded_file:

            if uploaded_file.type == "text/plain":
                text = uploaded_file.read().decode("utf-8")

            elif uploaded_file.type == "application/pdf":
                pdf_reader = PyPDF2.PdfReader(uploaded_file)
                for page in pdf_reader.pages:
                    text += page.extract_text() or ""

    if st.button("Analyze Readability"):

        if not text.strip():
            st.toast("No text found to analyze", icon="⚠️")
            return

        analyzer = ReadabilityAnalyzer(text)
        metrics = analyzer.get_all_metrics()

        st.subheader("Results")

        st.plotly_chart(create_gauge(metrics["Flesch Reading Ease"], "Flesch Reading Ease", 0, 100))

        st.plotly_chart(create_gauge(metrics["Flesch-Kincaid Grade"], "Flesch-Kincaid Grade", 0, 20))

        st.plotly_chart(create_gauge(metrics["SMOG Index"], "SMOG Index", 0, 20))

        st.plotly_chart(create_gauge(metrics["Gunning Fog"], "Gunning Fog", 0, 20))

        st.plotly_chart(create_gauge(metrics["Coleman-Liau"], "Coleman-Liau Index", 0, 20))

    if st.button("⬅ Back to Dashboard"):
        go_to("Dashboard")

# ------------ Gauge --------
def create_gauge(value, title, min_val, max_val):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        title={'text': title},
        gauge={'axis': {'range': [min_val, max_val]}}
    ))
    fig.update_layout(height=250)
    return fig

# ---------- ADMIN ----------
def admin_dashboard():

    st.sidebar.title("Admin Menu")

    if st.sidebar.button("👥 View Users"):
        st.session_state.admin_view = "users"

    if st.sidebar.button("🔄 Refresh Data"):
        st.rerun()

    if st.sidebar.button("🚪 Logout"):
        go_to("Login")

    st.title("🛠 Admin Dashboard")

    users = get_all_users()

    if not users:
        st.warning("No users found")
        return

    st.subheader("Registered Users")

    st.dataframe(
        users,
        use_container_width=True
    )

    st.subheader("Unlock User Account")

    emails = [u[1] for u in users]

    selected_email = st.selectbox("Select User Email", emails)

    if st.button("🔓 Unlock Account"):

        update_login_attempts(selected_email, 0, None)

        st.success("Account unlocked successfully")

# --------- forget page --------
def forgot_page():
    st.title("🔑 Forgot Password")

    email = st.text_input("Enter your registered email")

    method = st.radio(
        "Select Recovery Method",
        ["OTP Verification", "Security Question"]
    )

    if st.button("Continue"):

        user = get_user_by_email(email)

        if not user:
            st.toast("Email not registered", icon="❌")
            return

        st.session_state.pending_email = email

        # ⭐ Branch Logic
        if method == "OTP Verification":

            otp = generate_otp()
            send_otp_email(email, otp)

            st.session_state.otp = otp
            st.session_state.otp_time = datetime.utcnow()
            st.session_state.flow = "forgot_otp"

            go_to("OTP")

        else:
            st.session_state.flow = "forgot_security"
            go_to("SecurityQuestion")

    if st.button("⬅ Back"):
        go_to("Login")

#----- security questions ---------
def security_question_page():
    st.title("🔐 Security Verification")

    user = get_user_by_email(st.session_state.pending_email)

    st.write(user[4])  # security_question column

    answer = st.text_input("Your Answer")

    if st.button("Verify Answer"):

      if verify_text(answer, user[5]):

        if st.session_state.get("flow") == "forgot_security":
            go_to("SetNewPassword")
        else:
            st.toast("Unexpected flow", icon="⚠️")

      else:
        st.toast("Incorrect answer", icon="❌")

#--------- new password page ---------
def set_new_password_page():
    st.title("🔁 Set New Password")

    password = st.text_input("New Password", type="password")

    if st.button("Update Password"):

        valid, msg = validate_password(password)
        if not valid:
            st.toast(msg, icon="⚠️")
            return

        user = get_user_by_email(st.session_state.pending_email)

        # ✅ Load existing history
        history = json.loads(user[8] or "[]")

        # ✅ Prevent reuse of old passwords
        for old_hash in history:
            if verify_text(password, old_hash):
                st.toast("Cannot reuse old password", icon="⚠️")
                return

        new_hash = hash_text(password)

        # ✅ Update password
        update_password(st.session_state.pending_email, new_hash)

        # ✅ Update history properly
        history.insert(0, new_hash)
        history = history[:PASSWORD_HISTORY_COUNT]

        update_password_history(
            st.session_state.pending_email,
            json.dumps(history)
        )

        st.toast("Password reset successful", icon="✅")

        # Cleanup
        st.session_state.token = None
        go_to("Login")

# ---------- ROUTER ----------
page = st.session_state.page

if page == "Signup":
    signup_page()
elif page == "Dashboard":
    dashboard_page()
elif page == "Reset":
    reset_page()
elif page == "Forgot":
    forgot_page()
elif page == "OTP":
    otp_page()
elif page == "SecurityQuestion":
    security_question_page()
elif page == "SetNewPassword":
    set_new_password_page()
elif page == "AdminDashboard":
    admin_dashboard()
elif page == "Readability":
    readability_page()
else:
    login_page()

In [7]:
from google.colab import userdata
import os

os.environ["JWT_SECRET_KEY"] = userdata.get("JWT_SECRET_KEY")
os.environ["EMAIL_ID"] = userdata.get("EMAIL_ID")
os.environ["EMAIL_APP_PASSWORD"] = userdata.get("EMAIL_APP_PASSWORD")
os.environ["ADMIN_EMAIL_ID"] = userdata.get("ADMIN_EMAIL_ID")
os.environ["ADMIN_PASSWORD"] = userdata.get("ADMIN_PASSWORD")

In [37]:
!streamlit run app.py &>/content/log.txt &

In [ ]:
!pkill streamlit
!pkill ngrok

In [ ]:
from pyngrok import ngrok
from google.colab import userdata
token=userdata.get("NGROK_AUTHTOKEN")
ngrok.set_auth_token(token)
print(ngrok.connect(8501))
